# Avaliação U3 v2023 (pouco relevantes com peso 0,2)

In [1]:
 # Importa bibliotecas
import numpy as np
import pandas as pd
import nltk
import logging
import re
import os
import gc
from cryptography.fernet import Fernet # criptografia
import base64 # para criptografia
from bm25 import BM25L
from preprocessing import preprocess
import metrics_pr20 as metrics
import warnings
import copy
import pymssql
import getpass
import requests
import json

In [2]:
crypt_key = os.getenv('CRYPT_KEY_SOLIC_TRAB')
if crypt_key is None:
    crypt_key = getpass.getpass('Chave criptográfica')

Chave criptográfica········


In [3]:
# NLTK
logging.info('')
logging.info('Downloading NLTK...')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\p_7178\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\p_7178\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
###################
# Carga dos dados #
###################

# Feedbacks
logging.info('')
logging.info('Lendo base de queries e feedbacks...')

queries_fb = pd.read_csv('../data/usp_feedback_20220714_com_ponto.csv', usecols=['query', 'user_feedback', 'extra_results'])
print(queries_fb.shape)
queries_fb                                                                 

(703, 3)


,query,user_feedback,extra_results
0,Solicito projeto de lei para dispensar Guia de...,"[{""id"": ""PL 3650/2021"", ""class"": ""r"", ""score"":...","[""PL 7264/2014""]"
1,Solicito alterar a Lei 12.249 de 11 de junho d...,"[{""id"": ""PL 1158/2022"", ""class"": ""pr"", ""score""...",NaN
2,Requeiro à Consultoria projeto para que 20% da...,"[{""id"": ""INC 2833/2012"", ""class"": ""i"", ""score""...","[""1968/2020"", ""795/2020"", ""10378/2017"", ""PEC 3..."
3,Solicito à douta conle proposta para aumentar ...,"[{""id"": ""PEC 340/2013"", ""class"": ""i"", ""score"":...","[""12324/2019"", ""2884/2019"", ""1206/2019"", ""7303..."
4,Criação de fundo para dar assistência às pesso...,"[{""id"": ""PL 1434/2003"", ""class"": ""r"", ""score"":...","[""12680/2019"", ""10404/2008"", ""PL 431/2020"", ""P..."
...,...,...,...
698,ACIDENTE DE TRABALHO POR COVID PROFISSIONAIS D...,"[{""id"": ""PL 1612/2020"", ""class"": ""r"", ""score"":...","[""8371/2020"", ""5354/2020"", ""4454/2020"", ""4006/..."
699,PENSÃO ESPECIAL ÓRFÃOS COVID,"[{""id"": ""PL 2544/2021"", ""class"": ""r"", ""score"":...","[""15858/2021"", ""6095/2021"", ""5089/2021"", ""3739..."
700,PENSÃO ESPECIAL LGBT\n,"[{""id"": ""PL 2200/2020"", ""class"": ""i"", ""score"":...","[""6888/2020"", ""2153/2019"", ""888/2018"", ""PL 52/..."
701,PRORROGAÇÃO ESTADO DE EMERGÊNCIA COVID,"[{""id"": ""INC 6112/2014"", ""class"": ""i"", ""score""...","[""2169/2021"", ""12210/2020"", ""9470/2020"", ""8129..."


In [5]:
# Proposições
logging.info('')
logging.info('Lendo base de proposições...')

sql_conn = pymssql.connect(DB_CONNECTION_STRING)

SELECT_PROPOSICOES = 'SELECT code, name, text_preprocessed FROM Ulysses_U3_proposicao where code <= 2329852 ORDER BY code'
base_proposicoes = pd.read_sql(SELECT_PROPOSICOES, sql_conn)
print(base_proposicoes.shape)
base_proposicoes

Senha SqlPro········


C:\Users\p_7178\AppData\Local\Temp\ipykernel_8376\4174796175.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  base_proposicoes = pd.read_sql(SELECT_PROPOSICOES, sql_conn)


(107002, 3)


,code,name,text_preprocessed
0,16357,PL 1165/1999,"['ofici', '1416', 'sf', 'brasili', '17', 'julh..."
1,19098,PL 3097/1992,"['comissa', 'constituica', 'justic', 'redaca',..."
2,20464,PL 3927/2000,"['comissa', 'trabalh', 'administraca', 'servic..."
3,20683,PL 4117/1998,"['camar', 'deputad', 'departament', 'comisso',..."
4,20857,PL 4395/1998,"['1', 'comissa', 'constituica', 'justic', 'red..."
...,...,...,...
106997,2329845,PL 1730/2022,"['projet', 'lei', '2022', 'sr', 'gutemberg', '..."
106998,2329846,PDL 260/2022,"['camar', 'deputad', 'projet', 'decret', 'legi..."
106999,2329849,INC 1449/2022,"['requeriment', 'indicaca', '2022', 'sr', 'car..."
107000,2329850,PL 1731/2022,"['camar', 'deputad', 'projet', 'lei', 'no_____..."


In [6]:
if not os.path.isfile('../data/usp_corpus_20220714_inteiroteor_OCR.csv'):
    base_proposicoes.to_csv('../data/usp_corpus_20220714_inteiroteor_OCR.csv', index=False)

In [7]:
base_proposicoes[base_proposicoes['code']==605347]

,code,name,text_preprocessed
59875,605347,PL 4239/1958,"['projet', 'n', '4', '239', '1958', 'defin', '..."


In [8]:
# Solicitações
#SELECT_SOLICITACOES = 'SELECT ' + (f'TOP {TOP_ST} ' if TOP_ST > 0 else '') + 'name, text_preprocessed FROM Ulysses_U3_solicitacao where code <= 1169671 ORDER BY code desc'

logging.info('')
logging.info('Lendo base de solicitações...')

base_solicitacoes = pd.read_csv('../data/solicitacoes_ate_code_1169671.csv')
print(base_solicitacoes.shape)
base_solicitacoes

(78579, 2)


,name,text_preprocessed
0,8086/2022,Z0FBQUFBQmtTeWxNbEpBNS1sZ1Rwem9TQktDeEdMdEdGQ3...
1,8085/2022,Z0FBQUFBQmtTeWxNOTZZY19iYmtTMDNiWVFIRDNQa0w4eE...
2,8081/2022,Z0FBQUFBQmtTeWxNR3dLUUZ3SlgyWlZReE91cnhzanlSSz...
3,8079/2022,Z0FBQUFBQmtTeWxNdU1UUjBnek5WSFhrT2d2Q0xBQ3lpaU...
4,8078/2022,Z0FBQUFBQmtTeWxNV0tKcFNSTEVycUtNYVhLeU1yUk9xOW...
...,...,...
78574,11/2003,Z0FBQUFBQmtTeWxFalZLMFgzVk5BZERJTXJzSk9UZVpzeV...
78575,10/2003,Z0FBQUFBQmtTeWxFRUtseDlUSE15dEhxVF93SENXY0ZVTT...
78576,9/2003,Z0FBQUFBQmtTeWxFU3pZekx2dEF3ZkE5SFlhdDJEQzhMOW...
78577,5/2003,Z0FBQUFBQmtTeWxFY2JLVmdWYTE5anFqZm9ITktROWlYal...


In [9]:
queries_fb['extra_results'].fillna('[]', inplace=True)

In [10]:
LEI_REGEX1 = re.compile(r'"LEI .*?"\s*,\s*')
LEI_REGEX2 = re.compile(r'\s*,?\s*"LEI .*?"')
ST_REGEX1 = re.compile(r'"(ST )?\s*\d.*?"\s*,\s*')
ST_REGEX2 = re.compile(r'"\s*,?\s*(ST )?\s*\d.*?"')
queries_fb['qt_extra_pr'] = queries_fb['extra_results'].str.replace(LEI_REGEX1, '', regex=True).replace(LEI_REGEX2, '', regex=True).replace(ST_REGEX1, '', regex=True).replace(ST_REGEX2, '', regex=True).apply(lambda txt: len(eval(txt)))

PR_REGEX1 = re.compile(r'"\w+ .*?"\s*,\s*')
PR_REGEX2 = re.compile(r'\s*,?\s*"\w+ .*?"')
queries_fb['qt_extra_st'] = queries_fb['extra_results'].str.replace('ST ', '').replace(PR_REGEX1, '', regex=True).replace(PR_REGEX2, '', regex=True).apply(lambda txt: len(eval(txt)))

In [11]:
################################
# Pré-tratamento dos feedbacks #
################################

logging.info('')
logging.info('Pré-tratamento dos feedbacks...')

In [12]:
# Cria as colunas user_feedback_pr e user_feedback_st (separa proposições e solicitações, pois estão juntas no user_feedback)
def separa_feedbacks(feedbacks):

    lst_feedback_pr = []
    lst_feedback_st = []

    for feedback in eval(feedbacks):
        if feedback['tipo'] == 'PR':
            lst_feedback_pr.append(feedback)
        elif feedback['tipo'] == 'ST':
            lst_feedback_st.append(feedback)
        else:
            logging.info('Erro de tipo de feedback.')
                
    return lst_feedback_pr, len(lst_feedback_pr), lst_feedback_st, len(lst_feedback_st)

In [13]:
queries_fb['user_feedback_pr'], queries_fb['count_feedback_pr'], queries_fb['user_feedback_st'], queries_fb['count_feedback_st'] = zip(*queries_fb['user_feedback'].apply(separa_feedbacks))
queries_fb = queries_fb.drop('user_feedback', axis=1)

In [14]:
# Altera tipos de colunas
queries_fb.user_feedback_pr = queries_fb.user_feedback_pr.astype(str)
queries_fb.user_feedback_st = queries_fb.user_feedback_st.astype(str)

In [15]:
# Separa queries_fb em queries_pr (para proposições) e queries_st (para solicitações)
# queries_pr = queries_fb.copy()[['query', 'user_feedback_pr', 'count_feedback_pr']]
queries_pr = queries_fb.copy()[['query', 'user_feedback_pr', 'count_feedback_pr', 'qt_extra_pr']] #Incluído a variável qt_extra_pr
queries_pr.columns = ['query', 'user_feedback', 'count_feedback', 'qt_extra']
# queries_st = queries_fb.copy()[['query', 'user_feedback_st', 'count_feedback_st']]
queries_st = queries_fb.copy()[['query', 'user_feedback_st', 'count_feedback_st', 'qt_extra_st']] #Incluído a variável qt_extra_st
queries_st.columns = ['query', 'user_feedback', 'count_feedback', 'qt_extra']

In [16]:
# deleta queries_fb para liberar espaço de memória
del queries_fb
gc.collect();

In [17]:
# Separa proposições relevantes
pr_relevantes = []
pr_pouco_relevantes = []
for i in range(len(queries_pr["user_feedback"])):
    relevantes = []
    pouco_relevantes = []
    for doc in eval(queries_pr["user_feedback"][i]):
        if doc['class'] == 'r':
            relevantes.append(doc['id'])
        elif doc['class'] == 'pr':
            pouco_relevantes.append(doc['id'])
            
    pr_relevantes.append(relevantes)
    pr_pouco_relevantes.append(pouco_relevantes)

queries_pr = queries_pr.assign(relevantes = pr_relevantes, pouco_relevantes = pr_pouco_relevantes)
queries_pr.shape

(703, 6)

In [18]:
# Separa solicitações relevantes
st_relevantes = []
st_pouco_relevantes = []
for i in range(len(queries_st["user_feedback"])):
    relevantes = []
    pouco_relevantes = []
    for doc in eval(queries_st["user_feedback"][i]):
        if doc['class'] == 'r':
            relevantes.append(doc['id'])
        elif doc['class'] == 'pr':
            pouco_relevantes.append(doc['id'])
            
    st_relevantes.append(relevantes)
    st_pouco_relevantes.append(pouco_relevantes)

queries_st = queries_st.assign(relevantes = st_relevantes, pouco_relevantes = st_pouco_relevantes)
queries_st.shape

(703, 6)

In [19]:
# Dropa queries sem proposições relevantes/poucoRelevantes e sem extra_results
for i in range(len(queries_pr.values)):
    # if str(queries_pr["relevantes"][i]) == "[]":
    if str(queries_pr["relevantes"][i]) == "[]" and str(queries_pr["pouco_relevantes"][i]) == "[]" and queries_pr["qt_extra"][i] == 0: #Filtro alterado para levar em consideração os registros das colunas relevantes e qt_extra que estiverem vazio ou valor 0
    # if queries_pr[(queries_pr.relevantes.str.len() == 0) & (queries_pr.qt_extra == 0)]:
        queries_pr = queries_pr.drop(i)

queries_pr = queries_pr.reset_index(drop=True)
queries_pr.shape

(672, 6)

In [20]:
# Dropa queries sem solicitações relevantes/poucoRelevantes e sem extra_results
for i in range(len(queries_st.values)):
    # if str(queries_st["relevantes"][i]) == "[]":
    if str(queries_st["relevantes"][i]) == "[]" and str(queries_st["pouco_relevantes"][i]) == "[]" and queries_st["qt_extra"][i] == 0: #Filtro alterado para levar em consideração os registros das colunas relevantes e qt_extra que estiverem vazio ou valor 0
        queries_st = queries_st.drop(i)

queries_st = queries_st.reset_index(drop=True)
queries_st.shape

(664, 6)

In [21]:
#################################
# Pré-tratamento de proposições #
#################################

logging.info('Pré-tratamento de proposições...')
proposicoes_preprocessed = base_proposicoes[["name", "text_preprocessed"]]
base_preprocessada_pr = [eval(doc) for doc in proposicoes_preprocessed["text_preprocessed"]]
base_preprocessada_pr[0][:5]

['ofici', '1416', 'sf', 'brasili', '17']

In [22]:
##################################
# Pré-tratamento de solicitações #|
##################################

logging.info('Pré-tratamento de solicitações...')

# Desencriptar as STs
net = Fernet(crypt_key.encode())
net2 = Fernet(crypt_key)
base_solicitacoes['decrypted_tokens'] = base_solicitacoes.text_preprocessed.apply(lambda x: net.decrypt(base64.b64decode(x)).decode('utf-8'))

# Processar as STs
solicitacoes_preprocessed = base_solicitacoes[["name", "decrypted_tokens"]]
base_preprocessada_st = [eval(doc) for doc in solicitacoes_preprocessed["decrypted_tokens"]]
base_preprocessada_st[0][:5]

['solicit', 'avaliaca', 'critic', 'minut', 'pl']

In [23]:
###########
# Logging #
###########

logging.info('')
logging.info(f'Base de feedbacks (proposições) após o pré-tratamento: Shape = {queries_pr.shape}')
logging.info(f'Base de feedbacks (solicitações) após o pré-tratamento: Shape = {queries_st.shape}')

logging.info('')
logging.info(f'Tamanho da base de proposições a carregar no modelo BM25L: {len(base_preprocessada_pr)}')
logging.info(f'Tamanho da base de solicitações a carregar no modelo BM25L: {len(base_preprocessada_st)}')

In [24]:
###################
# Carrega modelos #
###################

# Carrega proposições
logging.info('')
logging.info('Carregando modelo BM25L (proposições)...')
bm25L_pr = BM25L(base_preprocessada_pr)

# Carrega solicitações
logging.info('Carregando modelo BM25L (solicitações)...')
bm25L_st = BM25L(base_preprocessada_st)

In [25]:
def expand_query(query):
    url_expand_query = 'https://expand-query-api.camara.leg.br'
    session = requests.Session()
    session.trust_env = False
    resp = session.post(url_expand_query, json={"query": query})
    if (resp.status_code == 200):
        return json.loads(resp.content)["query"]
    return query

In [26]:
%%time

expansion = True

############################################################################
# Calcula scores e lambdas para todos os documentos, para todas as queries #
############################################################################

MAX_K = 12 # Define max(k) para o cálculo das métricas precision@k, recall@k

# Define cut e delta

# EXECUTAR PRIMEIRO cut = 1.0 e delta = 1.0 (baseline)
# cut_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# delta_list = [0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0]

# Cut e delta para proposições
cut_pr = 0.4 #0.6
delta_pr = 0.1 #0.7

# Cut e delta para solicitações
cut_st = cut_pr
delta_st = delta_pr

# Calcula os scores e lambdas 

labels_pr = []
labels_st = []

for i in range(2):
    
    # inicializa os ponteiros

    if i == 0: # proposições
        logging.info('')
        logging.info('Calculando scores e lambdas das proposições, para todas as queries...')
                
        # hiperparâmetros do modelo
        cut = cut_pr
        delta = delta_pr

        # estruturas de dados
        labels_all = labels_pr
        queries_fb = queries_pr
        bm25L = bm25L_pr
        preprocessed_base = proposicoes_preprocessed

    else: # solicitações
        
        logging.info('')
        logging.info('Calculando scores e lambdas das solicitações, para todas as queries...')
                
        # hiperparâmetros do modelo
        cut = cut_st
        delta = delta_st

        # estruturas de dados
        labels_all = labels_st
        queries_fb = queries_st
        bm25L = bm25L_st
        preprocessed_base = solicitacoes_preprocessed

        # processa os cálculos

    n_queries = len(queries_fb)
    prev_progress_msg = ''
        
    for doc in range(n_queries):
        base_teste = queries_fb.iloc[doc]
        base_treino = queries_fb.drop(doc)
                
        arr_teste = base_teste.to_numpy()
        X, y = arr_teste[0], arr_teste[1] # X = queries; y = lista de feedbacks
        if expansion:
            X = expand_query(X)
        tokenized_query = preprocess(X) # tokeniza a query
                
        # calcula os scores para todos os documentos da base, com relação à query Q = tokenized_query
        if i == 0: # proposições
            scores = bm25L.get_partial_score(tokenized_query, preprocessed_base["text_preprocessed"])
        else: # solicitações
            scores = bm25L.get_partial_score(tokenized_query, preprocessed_base["decrypted_tokens"])
        # abaixo, normaliza os scores
        scores_norm = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))

        if cut == 1.0:
            
            # Para cut == 1.0 não calcula os lambdas, pq a similaridade entre queries não pode ser superior a 1.0
                    
            top_n = bm25L.get_top_n_score(scores_norm, preprocessed_base, n=MAX_K) # pega os MAX_K primeiros scores
                    
        else:
            
            # Aqui, calcula o lambda para cut < 1.0
                    
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                lambdas = bm25L.lambda_calc(base_treino, X, cut=cut, delta=delta)
                
            # Altera os scores para considerar o lambda (bônus para documentos marcados como relevantes em queries similares)
                
            new_scores = bm25L.lambda_update(scores_norm, lambdas, preprocessed_base)
            top_n = bm25L.get_top_n_score(new_scores, preprocessed_base, n=MAX_K)
                
        # armazena, em labels_all, os ids dos documentos que receberam os maiores scores (top_n=12) 
        # relacionados à query corrente 
        labels_all.append(copy.deepcopy(top_n))
                
        # progresso dos cálculos
        pct = int((doc/n_queries)*100)
        if pct in [25, 50, 75, 95]:
            progress_msg = f'Progresso: {pct}%'
            if progress_msg != prev_progress_msg: 
                logging.info(progress_msg)
                prev_progress_msg = progress_msg

# limpa os ponteiros para evitar erros no restante do código
cut = None
delta = None
labels_all = None
queries_fb = None
bm25L = None
preprocessed_base = None

CPU times: total: 46min 31s
Wall time: 47min 52s


In [27]:
import datetime
db_date = datetime.datetime.now()
db_date

datetime.datetime(2023, 5, 18, 10, 40, 26, 724751)

In [28]:
#####################
# Calcular métricas #
#####################

logging.info('')
logging.info('Calculando métricas...')

# Proposições
resultados_pr = {}
resultados_pr['data'] = db_date
resultados_pr['tipo_doc'] = 'proposições'
resultados_pr['cut'] = [cut_pr,]
resultados_pr['delta'] = [delta_pr,]

# Solicitações 
resultados_st = {}
resultados_st['data'] = db_date
resultados_st['tipo_doc'] = 'solicitações'
resultados_st['cut'] = [cut_st,]
resultados_st['delta'] = [delta_st,]

In [29]:
print(resultados_pr)
print(resultados_st)

{'data': datetime.datetime(2023, 5, 18, 10, 40, 26, 724751), 'tipo_doc': 'proposições', 'cut': [0.4], 'delta': [0.1]}
{'data': datetime.datetime(2023, 5, 18, 10, 40, 26, 724751), 'tipo_doc': 'solicitações', 'cut': [0.4], 'delta': [0.1]}


In [30]:
queries_pr.shape, queries_st.shape

((672, 6), (664, 6))

In [31]:
#############
# Precision #
#############

results_precision_pr = metrics.calcula_precision(queries_pr, labels_pr, MAX_K)
results_precision_st = metrics.calcula_precision(queries_st, labels_st, MAX_K)

for k in results_precision_pr:
    resultados_pr['p@' + str(k)] = [results_precision_pr[k],]

for k in results_precision_st:
    resultados_st['p@' + str(k)] = [results_precision_st[k],]

In [32]:
##########
# Recall #
##########

results_recall_pr = metrics.calcula_recall(queries_pr, labels_pr, MAX_K)
results_recall_st = metrics.calcula_recall(queries_st, labels_st, MAX_K)

for k in results_recall_pr:
    resultados_pr['r@' + str(k)] = [results_recall_pr[k],]

for k in results_recall_st:
    resultados_st['r@' + str(k)] = [results_recall_st[k],]

In [33]:
#########################################################
# Average Precision (AP) e Mean Average Precision (MAP) #
#########################################################

lista_avg_precision_pr = metrics.calcula_lista_avg_precision(queries_pr, labels_pr)
avg_precision_pr = metrics.calcula_avg_precision(queries_pr, lista_avg_precision_pr)
resultados_pr['map'] = [np.mean(avg_precision_pr),] # calcula MAP (mean average precision) para proposições

lista_avg_precision_st = metrics.calcula_lista_avg_precision(queries_st, labels_st)
avg_precision_st = metrics.calcula_avg_precision(queries_st, lista_avg_precision_st)
resultados_st['map'] = [np.mean(avg_precision_st),] # calcula MAP (mean average precision) para solicitações

In [34]:
################################################
# R-Precision (RP) e Average R-Precision (ARP) #
################################################

lista_r_precision_pr = metrics.calcula_r_precision(queries_pr, labels_pr)
resultados_pr['r-precision'] = [np.mean(lista_r_precision_pr),] # calcula ARP para proposições

lista_r_precision_st = metrics.calcula_r_precision(queries_st, labels_st)
resultados_st['r-precision'] = [np.mean(lista_r_precision_st),] # calcula ARP para solicitações

In [35]:
#####################################################
# Reciprocal Rank (RR) e Mean Reciprocal Rank (MRR) #
#####################################################

RR_lista_pr = metrics.calcula_rr(lista_avg_precision_pr)
resultados_pr['rr'] = [np.mean(RR_lista_pr),] # calcula MRR

RR_lista_st = metrics.calcula_rr(lista_avg_precision_st)
resultados_st['rr'] = [np.mean(RR_lista_st),] # calcula MRR

In [36]:
for item in resultados_pr:
    print('\t', item, resultados_pr[item])
print('\n')
print('resultados Solicitações:')
for item in resultados_st:
    print('\t', item, resultados_st[item])

	 data 2023-05-18 10:40:26.724751
	 tipo_doc proposições
	 cut [0.4]
	 delta [0.1]
	 p@1 [0.619345238095237]
	 p@2 [0.5748511904761912]
	 p@3 [0.5375992063492057]
	 p@4 [0.509523809523811]
	 p@5 [0.48624999999999946]
	 p@6 [0.4656746031746031]
	 p@7 [0.4459183673469385]
	 p@8 [0.42756696428571495]
	 p@9 [0.4107804232804236]
	 p@10 [0.39392857142857124]
	 p@11 [0.3787337662337662]
	 p@12 [0.36426091269841293]
	 r@1 [0.13266939004535455]
	 r@2 [0.22985139971685375]
	 r@3 [0.3108215808952314]
	 r@4 [0.3802606111042171]
	 r@5 [0.43625447425855896]
	 r@6 [0.48801908439648356]
	 r@7 [0.5389846323934583]
	 r@8 [0.5788327668815573]
	 r@9 [0.6159689417231082]
	 r@10 [0.6493374351194197]
	 r@11 [0.6819328934631362]
	 r@12 [0.7141513942184636]
	 map [0.6086357860114769]
	 r-precision [0.4708244818484185]
	 rr [0.6725966080876795]


resultados Solicitações:
	 data 2023-05-18 10:40:26.724751
	 tipo_doc solicitações
	 cut [0.4]
	 delta [0.1]
	 p@1 [0.5659638554216859]
	 p@2 [0.49201807228915695]
	 p

In [37]:
# #####################################
# # Gravar métricas no banco de dados #
# #####################################

# SALVAR_METRICAS_BD = True

# if SALVAR_METRICAS_BD:

#     logging.info('')
#     logging.info('Salvando métricas no BD...')

#     # Primeiro, grava o monitoramento na tabela Ulysses_Monitoramento
#     INSERT_MONITORING = 'INSERT Ulysses_Monitoramento ' \
#         '(nom_Sistema, txt_Modelo, arq_Dataset_Teste, txt_Mlflow_Uri, dat_execucao, json_indicadores) ' \
#         'VALUES (' \
#         f'\'U3\', \'U3-BM25L\', null, null, \'{db_date}\', null)'
    
#     # Monta o bloco de SQLs
#     sql_block = ''
#     sql_block += 'BEGIN\n'
#     sql_block += 'DECLARE @ident int = 0\n'
#     sql_block += f'{INSERT_MONITORING}\n'
#     sql_block += 'SELECT @ident = @@IDENTITY\n'

#     # Depois, insere as métricas na tabela Ulysses_MonitoramentoResultados
#     lst_metrics = ['u3-map-pr', 'u3-arp-pr', 'u3-mrr-pr', 'u3-map-st', 'u3-arp-st', 'u3-mrr-st']
#     lst_values = [resultados_pr["map"], resultados_pr["r-precision"], resultados_pr["rr"], resultados_st["map"], resultados_st["r-precision"], \
#         resultados_st["rr"]]
#     #
#     for metric, value in zip(lst_metrics, lst_values):

#         INSERT_METRICS = 'INSERT Ulysses_MonitoramentoResultados ' \
#             '(seq_Monitoramento, nom_Indicador, vlr_Indicador) ' \
#             f'SELECT @ident, \'{metric}\', {value[0]}' 
        
#         sql_block += f'{INSERT_METRICS}\n'
#     #
#     sql_block += 'END'
#     #
#     mssql_hook.run(sql_block) # executa bloco de comandos SQL

In [38]:
######################
# Mostrar resultados #
######################

# Proposições

print('### Resultados para proposições #############################')
print(f'Precision (P@12): {resultados_pr["p@12"]}')
print(f'Recall (R@12): {resultados_pr["r@12"]}')
print(f'Average R-Precision (ARP): {resultados_pr["r-precision"]}')
print(f'Mean Average Precision (MAP): {resultados_pr["map"]}')
print(f'Mean Reciprocal Rank (MRR): {resultados_pr["rr"]}')
print('#############################################################')

# Solicitações

print('### Resultados para solicitações ############################')
print(f'Precision (P@12): {resultados_st["p@12"]}')
print(f'Recall (R@12): {resultados_st["r@12"]}')
print(f'Average R-Precision (ARP): {resultados_st["r-precision"]}')
print(f'Mean Average Precision (MAP): {resultados_st["map"]}')
print(f'Mean Reciprocal Rank (MRR): {resultados_st["rr"]}')
print('#############################################################')

### Resultados para proposições #############################
Precision (P@12): [0.36426091269841293]
Recall (R@12): [0.7141513942184636]
Average R-Precision (ARP): [0.4708244818484185]
Mean Average Precision (MAP): [0.6086357860114769]
Mean Reciprocal Rank (MRR): [0.6725966080876795]
#############################################################
### Resultados para solicitações ############################
Precision (P@12): [0.28757530120481944]
Recall (R@12): [0.7061376533907266]
Average R-Precision (ARP): [0.3994310737378868]
Mean Average Precision (MAP): [0.5577393773423044]
Mean Reciprocal Rank (MRR): [0.633928354109077]
#############################################################


In [39]:
######################################
# Salvar resultados detalhados no S3 #
######################################

# logging.info('')
# logging.info('Salvando métricas no S3...')

# # dataframes
# df_results_pr = pd.DataFrame(resultados_pr)
# df_results_st = pd.DataFrame(resultados_st)

# # juntar dataframes
# df_results = pd.concat([df_results_pr, df_results_st], ignore_index=True)

# # apresentar resultados detalhados
# logging.info('')
# logging.info('Abaixo, resultados detalhados:')
# logging.info(df_results.transpose())

# # Ler CSV anterior em S3 para fazer o append dos resultados desta execução
# s3_hook = S3Hook(aws_conn_id='s3_secid')
# s3_bucket = 'secid.ulysses'
# s3_key = 'u3/metricas/data/prd/metricas_bm25l.csv'

# if s3_hook.check_for_key(key=s3_key, bucket_name=s3_bucket):
#     # CSV já existe, concatena com o resultado desta execução
#     file_csv = s3_hook.download_file(key=s3_key, bucket_name=s3_bucket) # faz o download
#     prev_df = pd.read_csv(file_csv, encoding='utf-8')
#     df_results = pd.concat([prev_df, df_results], ignore_index=True)
#     # Salva CSV com os novos resultados
#     df_results.to_csv(file_csv, encoding='utf-8', index=False)
#     # Salva CSV em S3
#     s3_hook.load_file(filename=file_csv, key=s3_key, bucket_name=s3_bucket, replace=True)
# else:
#     # Cria arquivo CSV
#     with NamedTemporaryFile("w") as f_csv:
#         # Salva CSV com os novos resultados
#         df_results.to_csv(f_csv.name, encoding='utf-8', index=False)
#         # Salva CSV em S3
#         s3_hook.load_file(filename=f_csv.name, key=s3_key, bucket_name=s3_bucket, replace=True)

# return 0

## Análise qualitativa

In [40]:
def top_pr(n, query_index, cut, delta):
    return top(n, query_index, queries_pr, proposicoes_preprocessed, "text_preprocessed", bm25L_pr, cut=cut, delta=delta)

def top_st(n, query_index, cut, delta):
    return top(n, query_index, queries_st, solicitacoes_preprocessed, "decrypted_tokens", bm25L_st, cut=cut, delta=delta)

def top(n, query_index, queries, preprocessed_base, base_col, bm25L, cut, delta):
    base_teste = queries.iloc[query_index]
    base_treino = queries.drop(query_index)
    arr_teste = base_teste.to_numpy()
    X, y = arr_teste[0], arr_teste[1] # X = queries; y = lista de feedbacks
    qry = X
    expanded_qry = None
    if expansion:
        expanded_qry = expand_query(X)
        if X == expanded_qry:
            expanded_qry = None
        else:
            X = expanded_qry
    tokenized_query = preprocess(X) # tokeniza a query            
    scores = bm25L.get_partial_score(tokenized_query, preprocessed_base[base_col])
    scores_norm = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
    if cut == 1.0:
        top_n = bm25L.get_top_n_score(scores_norm, preprocessed_base, n=n)
        lambdas = {}
    else:
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            lambdas = bm25L.lambda_calc(base_treino, X, cut=cut, delta=delta)
        new_scores = bm25L.lambda_update(scores_norm, lambdas, preprocessed_base)
        top_n = bm25L.get_top_n_score(new_scores, preprocessed_base, n=n)
    return qry, expanded_qry, lambdas, top_n

In [41]:
cut = 0.4 #0.6 na versão anterior
delta = 0.1 #0.7 na versão anterior

In [42]:
%%time
for i in range(len(queries_pr)):
    qry, expanded_qry, lambdas, top_n = top_pr(12, i, cut=cut, delta=delta)
    if expanded_qry:
        print('Proposições similares:', qry, expanded_qry, lambdas, top_n)
        print()

Proposições similares: Solicito uma emenda ao PL 3041/2021 no sentido de criar mecanismos de acompanhamento e avaliação do Programa de Apoio ao Desenvolvimento Tecnológico da Indústria de Semicondutores - PADIS, instituído pela Lei nº 11.484, de 31 de maio de 2007.  Solicito uma emenda ao PL 3041/2021 no sentido de criar mecanismos de acompanhamento e avaliação do Programa de Apoio ao Desenvolvimento Tecnológico da Indústria de Semicondutores - PADIS, instituído pela Lei nº 11.484, de 31 de maio de 2007.  Institui a Semana Nacional das Práticas Integrativas e Complementares em Saúde. {} ['INC 3039/2017', 'INC 3038/2017', 'PL 719/2015', 'PL 2156/2007', 'PL 3130/2015', 'PL 4944/2019', 'PL 3041/2021', 'PL 3042/2021', 'PL 3863/2015', 'PL 2821/2019', 'INC 211/2019', 'PL 3682/2019']

Proposições similares: Solicito Parecer, pela aprovação e adequação orçamentária, do PLP 108/2021. Deve conter substitutivo com correção do MEI e do Simples pelo IPCA, a partir da última atualização.  Solicito P

In [43]:
%%time
for i in range(len(queries_st)):
    qry, expanded_qry, lambdas, top_n = top_st(12, i, cut=cut, delta=delta)
    if expanded_qry:
        print('Solicitações similares:', qry, expanded_qry, lambdas, top_n)
        print()

Solicitações similares: Solicito uma emenda ao PL 3041/2021 no sentido de criar mecanismos de acompanhamento e avaliação do Programa de Apoio ao Desenvolvimento Tecnológico da Indústria de Semicondutores - PADIS, instituído pela Lei nº 11.484, de 31 de maio de 2007.  Solicito uma emenda ao PL 3041/2021 no sentido de criar mecanismos de acompanhamento e avaliação do Programa de Apoio ao Desenvolvimento Tecnológico da Indústria de Semicondutores - PADIS, instituído pela Lei nº 11.484, de 31 de maio de 2007.  Institui a Semana Nacional das Práticas Integrativas e Complementares em Saúde. {} ['23181/2019', '8584/2019', '25151/2019', '5079/2011', '14865/2007', '14369/2007', '6520/2008', '6694/2010', '5105/2009', '7830/2010', '6154/2020', '15838/2021']

Solicitações similares: Solicito Parecer, pela aprovação e adequação orçamentária, do PLP 108/2021. Deve conter substitutivo com correção do MEI e do Simples pelo IPCA, a partir da última atualização.  Solicito Parecer, pela aprovação e adequ